In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import urllib

In [3]:
import re

In [5]:
def get_last_page_number(domain, pagination_attrs = {'class':'pagination'}, has_next = True):
    html = requests.get(DOMAIN).text
    soup = BeautifulSoup(html, 'lxml')
    
    if has_next:
        last_index = -2
    else:
        last_index = -1
    
    last_page_number = int(soup.find(attrs = pagination_attrs).find_all('a')[last_index].get_text())
    
    return last_page_number

In [6]:
def get_links(domain, last_page_number):
    links_site = []
    
    for page in range(1, last_page_number):
        url = '{}/page/{}/'.format(domain, page + 1)

        html = requests.get(url).text
        soup = BeautifulSoup(html, 'lxml')
        current_links = [a.get('href') for a in soup.find_all('a')]
        current_links_site = list(filter(lambda x: x.startswith(DOMAIN), current_links))
        links_site += current_links_site
    
    links_site = list(set(links_site))
    links_site.sort()
    return links_site

In [17]:
def filter_links_to_movies(links, movie_pattern):
    links_to_movies = list(filter(lambda x: re.search(movie_pattern, x), 
                                  links))
    links_to_movies.sort()
    return links_to_movies

In [23]:
def get_movie_titles(links_to_movies, encoding = 'windows-1252', decoding = 'windows-1251'):
    movies_titles = []
    
    for url in links_to_movies:
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'lxml')
        try:
            title = soup.find('title').get_text().encode(encoding).decode(decoding)
        except UnicodeEncodeError:
            continue
        movies_titles.append(title)
    
    return movies_titles

------

In [8]:
DOMAIN = 'http://bobfilm1.net'

last_page_bobfilm1 = get_last_page_number(DOMAIN)
links_bobfilm1 = get_links(DOMAIN, last_page_bobfilm1)
links_movies_bobfilm1 = filter_links_to_movies(links_bobfilm1, '^http://bobfilm1\.net/\d')
titles = get_movie_titles(links_movies_bobfilm1)

In [ ]:
DOMAIN = 'http://kinogo.club'

last_page_kinogo = get_last_page_number(DOMAIN, {'class':'bot-navigation'})
links_kinogo = get_links(DOMAIN, last_page_kinogo)
links_movies_kinogo = filter_links_to_movies(links_kinogo, '^http://kinogo\.club/\d')
titles_kinogo = get_movie_titles(links_movies_kinogo)

In [27]:
# len(links)
# links = list(set(links))
# links.sort()
# links[:10]
# len(links_movies)
# len(titles)
# titles[:10]